## Навигация
- **[Функции обработки данных](#Функции-обработки-данных)**
- **[Трансформация данных](#Трансформация-данных)**
- **[Сборка для Импорт](#Сборка-для-Импорт)**
- **[Сборка для Экспорт](#Сборка-для-Экспорт)**
- **[Сборка основного датасета](#Сборка-основного-датасета)**
- **[Сохраняем данные в БД](#Сохраняем-данные-в-БД)**

In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path 

from tqdm import tqdm
import warnings
warnings.simplefilter('ignore') 

# Для автоматического закрытия курсора
from contextlib import closing

import psycopg2
from psycopg2 import Error
from sqlalchemy import create_engine

import json
from datetime import datetime

from dotenv import load_dotenv

# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)

In [2]:
dotenv_path = './.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path)

In [3]:
dict_postgres_cred = {'user': os.getenv('USER_NAME_PG'),
                      'password': os.getenv('PASSWORD_PG'),
                      'host': os.getenv('HOST_PG'),
                      'port': os.getenv('PORT_PG'),
                      'database': os.getenv('DATABASE_PG')}

In [4]:
# Инициализация подключений для работы с БД
engine = psycopg2.connect(user=dict_postgres_cred['user'],
                          password=dict_postgres_cred['password'],
                          host=dict_postgres_cred['host'],
                          port=dict_postgres_cred['port'],
                          database=dict_postgres_cred['database'])
conn = create_engine('postgresql://{}:{}@{}:{}/{}'
                     .format(dict_postgres_cred['user'], dict_postgres_cred['password'], 
                             dict_postgres_cred['host'], dict_postgres_cred['port'], dict_postgres_cred['database']))


In [5]:
# Функция для переименования всех столбцов
def rep(name):
    return str(name) 

def rep_2(name):
    return str(name).replace('-', '_') 

def rep_3(name):
    return str(name).replace(' ', '_') 

In [6]:
# Справочник из БД с единицами измерения
query_deizm = f"""
        SELECT * FROM izm
        ORDER BY id ASC 
"""

df_deizm = pd.read_sql(query_deizm, con=engine)
df_deizm.rename(columns={'code': 'qty_unit_code','name_rus': 'units_value'}, inplace=True)
df_deizm = df_deizm[['qty_unit_code', 'units_value']]

In [7]:
dict_deizm = {}
# Заполняем словарь для перехода от ед. изм. itc к общепринятым
dict_deizm['Tons'] = 'килограмм'
dict_deizm['без размерности'] = 'без размерности'
dict_deizm['No quantity'] = 'без размерности'
dict_deizm['Pairs'] = 'пара'
dict_deizm['Units'] = 'штука'
dict_deizm['Cubic meters'] = 'кубический метр'
dict_deizm['1000 meters'] = 'метр'
dict_deizm['1000 square meters'] = 'квадратный метр'
dict_deizm['Carats'] = 'карат'
dict_deizm['Mega watt hours'] = '1000 кВаттЧас'
dict_deizm['Thousands'] = '1000 штук'
dict_deizm['Mixed'] = 'без размерности'  
dict_deizm['Number of packages'] = '1 пачка'
dict_deizm['Dozen'] = 'дюжина'

In [8]:
# Словаврь для партнера
query_country_add = f"""

SELECT code, name_itc FROM add

"""
df_country_add = pd.read_sql(query_country_add, con=engine)

# Подготовим колонку для мержа
df_country_add['test_name'] = df_country_add.name_itc.apply(lambda x: x.replace(',', ''))

# Чистим данные
df_country_add.drop_duplicates(subset=['name_itc'], inplace=True)

In [9]:
# Словарь для присовения кодов Репортеру
dict_partner_code = {}
dct_itc_and_test_name = {}
for code_itc, name_test in zip(list(df_country_add.code), list(df_country_add.test_name)):
    if name_test not in dict_partner_code:
        dict_partner_code[name_test] = code_itc
    else:
        continue
# Для сопоставления наших названий стран и стран из itc
for itc_name, name_test in zip(list(df_country_add.name_itc), list(df_country_add.test_name)):
    if name_test not in dct_itc_and_test_name:
        dct_itc_and_test_name[name_test] = itc_name
    else:
        continue

In [10]:
def get_need_tnved_code() -> dict:
    """
    Возвращает набор необходимых кодов ТНВЭД

    :return: словарь с кодами (пока что на 6 знаках 'code_6')
    """
   
    dict_return = {'code_6': []}
    with conn.connect() as connection:
        rez_query = connection.execute("""SELECT DISTINCT(LEFT(code,6)) 
                                               FROM ed 
                                               WHERE type = 10 AND prod_type = 'apk' AND LEFT(code, 2)::int > 24""")
    for code in rez_query.fetchall():
        code_cleare = code[0]
        dict_return['code_6'].append(code_cleare)
    return dict_return

dict_need_tnved_code_apk = get_need_tnved_code()

In [11]:
# Года с атрибутом Mirror data
# Их мы исключаем при сборке
if os.path.exists('json_mirror_data.json'):
    with open('json_mirror_data.json', encoding='utf-8') as fl:
        js = json.load(fl)
else:
    js = {}

In [12]:
js

{'Albania': ['2023']}

# Функции обработки данных

In [13]:
def trade_value_build(path_values: str, type_operation: int) -> pd.DataFrame:
    """
    Собирает все файлы в один датафрейм для переданного направления торговли
    по переданному пути
    
    :param path_values: путь к файлам для TRADE_VALUE
    
    :param type_operation: тип операции (импорт - 1 или экспорт - 2)
    
    :return: очищенный датафрейм с данными по TRADE_VALUE
    """
    # Объект типа Path для TRADE_VALUE
    way_pah_values = Path(path_values)

    # Пустой датафрейм для сборки всех значений trade_value
    void_df_value = pd.DataFrame()

    # Сбор файлов trade_value
    for i in tqdm(way_pah_values.glob("**/Tra*.txt")):

        if flag_and_in_reporter_name:
            # Если в названии репортера присутствует _and_ : Antigua_and_Barbuda
            reporter = ' and '.join(str(i).split('_between_')[1].split('_and_')[:2]).replace('_', ' ').replace('  ', ' ')
            partner = ' and '.join(str(i).split('_between_')[1].split('_and_')[2:]).replace('.txt', '').replace('_', ' ').replace('  ', ' ')  
        else:
            # Если репортер без _and_ : Cabo_Verde
            reporter = str(i).split('_between_')[1].split('_and_')[0].replace('_', ' ').replace('  ', ' ')
            partner = ' and '.join(str(i).split('_between_')[1].split('_and_')[1:]).replace('.txt', '').replace('_', ' ').replace('  ', ' ')

        # Если есть зеркальные года, сохраняеим их
        lst_query = js[reporter] if reporter in js else []

        temp_df = pd.read_table(i, dtype={'Product code': 'str'})

        # Получаем нужное количество колонок
        need_max_year = max([int(i.split(' in ')[-1]) for i in temp_df.columns.tolist() if ' in ' in i])
        for temp in range(len(list(temp_df.columns))):
            if f'-Value in {need_max_year}' in list(temp_df.columns)[temp]:
                temp_number_columns = temp + 1
                break
        if partner == 'World':
            temp_df = temp_df.drop(columns=list(temp_df.iloc[:, 2:16].columns)).iloc[:, 0:14]
        else:
            temp_df = temp_df.iloc[:,0:temp_number_columns]

        # Расплавляем датасет
        temp_df = temp_df.melt(id_vars=['Product code', 'Product label'])
        temp_df.rename(columns = rep_3, inplace=True)
        
        # Отсекаем не нужное
        temp_df = temp_df.query('value > 0 and Product_code != "TOTAL"')
        temp_df = temp_df[
            (temp_df['Product_code'].str[:2].astype(int) <= 24) |
            (temp_df['Product_code'].str[:2].astype(int) == 31) |
            (temp_df['Product_code'].str[:6].isin(dict_need_tnved_code_apk['code_6']))
            ]
        temp_df.fillna(0, inplace=True)

       
        # Добавляем столбцы
        temp_df = temp_df.assign(reporter_country = reporter ,partner_country = partner, trade_flow_code = type_operation, 
                                classification='HS', update_date=datetime.now().strftime('%Y-%m-%d'))
        temp_df['year_transaction'] = temp_df.variable.apply(lambda x: x.split(' in ')[1])
        temp_df = temp_df.query('year_transaction not in @lst_query')
        temp_df['period'] = temp_df.year_transaction.apply(lambda x: datetime.strptime('01-01-' + x, '%d-%m-%Y'))
        temp_df['aggregate_level'] = 6
        temp_df['flag'] = 0
        temp_df['plus'] = 0
        temp_df['load_mark'] = 1
        temp_df['value'] = temp_df.value.mul(1000)
        void_df_value = pd.concat((void_df_value, temp_df))

    # Создаем колонку для мержа
    void_df_value['test_name'] = void_df_value.partner_country.apply(lambda x: x)
    # Удаляем лишнее
    void_df_value.drop(columns='variable', inplace=True)

    # Мержим наш void_df_value с COUNTRY_ADD
    df_merge = void_df_value.merge(df_country_add, how='left',  on='test_name')

    df_merge.rename(columns={'code': 'partner_code'}, inplace=True)
    # В зависимости от страны проставляем код
    df_merge['reporter_code'] = df_merge.reporter_country.apply(lambda x: dict_partner_code[x])
    
    return df_merge

In [14]:
def check_not_download_file(type_flow: str, path_values: str, path_quantities: str) -> dict:
    """
    Проверяет, всели файлы по Quantities мы скачали исходя из файлов TRADE_VALUE
    Если были скачены не все, функция вернет словарь с партнерами, которые отсутствуют
    
    :param type_flow: тип операции (импорт или экспорт)
    
    :param path_values: путь к файлам для TRADE_VALUE
    
    :param path_quantities: путь к файлам для Quantities
    
    :return: словарь со странами, которые мы не скачали, если такие есть
    """

    # Проверка не скаченных файлов
    # Если таких нет, докачайте их
    dct_error_country = {'type_flow': type_flow,
                         'reporter_name': '', 'list_partner': []}


    l_values = [str(i).split('\\')[-1] for i in Path(path_values).glob('**/Tra*.txt')]
    l_quantities = [str(i).split('\\')[-1] for i in Path(path_quantities).glob('**/Tra*.txt')]
    for i in l_values:
        if i not in l_quantities:
            # Получаем название репортера и партнера
            if flag_and_in_reporter_name:
                # Если в названии репортера присутствует _and_ : Antigua_and_Barbuda
                reporter = ' and '.join(str(i).split('_between_')[1].split('_and_')[:2]).replace('_', ' ').replace('  ', ' ')
                partner = ' and '.join(str(i).split('_between_')[1].split('_and_')[2:]).replace('.txt', '').replace('_', ' ').replace('  ', ' ')  
            else:
                # Если репортер без _and_ : Cabo_Verde
                reporter = str(i).split('_between_')[1].split('_and_')[0].replace('_', ' ').replace('  ', ' ')
                partner = ' and '.join(str(i).split('_between_')[1].split('_and_')[1:]).replace('.txt', '').replace('_', ' ').replace('  ', ' ')
            # Заполняем словарь не докаченных файлов
            dct_error_country['reporter_name'] = dct_itc_and_test_name[reporter]
            if dct_itc_and_test_name[partner] not in dct_error_country['list_partner']:
                dct_error_country['list_partner'].append(dct_itc_and_test_name[partner])


            print(f"{i} отсутствуют")
    return dct_error_country

In [15]:
def quantities_build(type_operation: str, path_values: str, path_quantities: str) -> pd.DataFrame:
    """
    Собирает все файлы в один датафрейм для переданного направления торговли
    по переданному пути
    
    :param type_operation: тип операции (импорт или экспорт)
    
    :param path_values: путь к файлам для TRADE_VALUE
    
    :param path_quantities: путь к файлам для Quantities
    
    :return: очищенный датафрейм с данными по Quantities
    """
    # Создаем объект типа Path
    way_pah_quantities = Path(path_quantities)

    # Пустой датафрейм для Quantities
    void_df_quantities = pd.DataFrame()

    dct_error = check_not_download_file(type_operation, path_values, path_quantities)
    # Сбор файлов Quantities
    for i in tqdm(way_pah_quantities.glob('**/Tra*.txt')):
        if flag_and_in_reporter_name:
            # Если в названии репортера присутствует _and_ : Antigua_and_Barbuda
            reporter = ' and '.join(str(i).split('_between_')[1].split('_and_')[:2]).replace('_', ' ').replace('  ', ' ')
            partner = ' and '.join(str(i).split('_between_')[1].split('_and_')[2:]).replace('.txt', '').replace('_', ' ').replace('  ', ' ')  
        else:
            # Если репортер без _and_ : Cabo_Verde
            reporter = str(i).split('_between_')[1].split('_and_')[0].replace('_', ' ').replace('  ', ' ')
            partner = ' and '.join(str(i).split('_between_')[1].split('_and_')[1:]).replace('.txt', '').replace('_', ' ').replace('  ', ' ')


        temp_df = pd.read_table(i, dtype={'Product code': 'str'})

        # Если был скачен не тот фалй (value вместо quantities)
        # То мы фиксируем его в словаре ошибок, для дальнейшей обработки
        if len([i for i in temp_df.columns.tolist() if '-Value in ' in i]) > 0:
            print(f'Ошибка в партнере: {partner}')
            if dct_itc_and_test_name[partner] not in dct_error['list_partner']:
                dct_error['list_partner'].append(dct_itc_and_test_name[partner])
            print(f"Удаляем {i}")    
            i.unlink()
        else:
            # Если есть зеркальные года, сохраняеим их
            lst_query = js[reporter] if reporter in js else []
            #Считаем нужное кол-во столбцов(максимальный и минимальный года)
            need_max_year = max([int(i.split(' in ')[-1]) for i in temp_df.columns.tolist() if ' in ' in i])
            need_min_year = min([int(i.split(' in ')[-1]) for i in temp_df.columns.tolist() if ' in ' in i])
            for temp in range(len(list(temp_df.columns))):
                if f'-Quantity in {need_max_year}' in list(temp_df.columns)[temp]:
                    temp_number_columns = temp + 2
                    break

            if partner == 'World':
                temp_df = temp_df.drop(columns=list(temp_df.iloc[:, 2:28].columns)).iloc[:, 0:26]
            else:
                temp_df = temp_df.iloc[:,0:temp_number_columns] #26

            # Расплавляем датасет
            temp_df = temp_df.melt(id_vars=['Product code', 'Product label'])
            temp_df.rename(columns = rep_3, inplace=True)

            # Отсекаем не нужное
            temp_df.fillna(0, inplace=True)
            temp_df = temp_df[(temp_df['value'] != 0) & (temp_df['Product_code'] != "TOTAL")]
            temp_df = temp_df[
                (temp_df['Product_code'].str[:2].astype(int) <= 24) |
                (temp_df['Product_code'].str[:2].astype(int) == 31) |
                (temp_df['Product_code'].str[:6].isin(dict_need_tnved_code_apk['code_6']))
                ]
            
            
            temp_df = temp_df.assign(reporter_country = reporter ,partner_country = partner)

            # Создаем два датасета, первый для названия измерений
            # Второй для самих измерений
            temp_df['bool_unit'] = temp_df.variable.apply(lambda x: True if '-Unit' in x else False)
            temp_df['bool_quantity'] = temp_df.variable.apply(lambda x: True if '-Unit' not in x else False)
            df_unit_tmp = temp_df.loc[temp_df.bool_unit]
            df_quantity_tmp = temp_df.loc[temp_df.bool_quantity]

            # Формируем лист из "якорей" для опеределения года в датасете с названиями измерений
            list_unit_from_key = []
            for i in list(df_unit_tmp.variable.unique()):
                if i.split('-U')[1] not in list_unit_from_key:
                    list_unit_from_key.append(i.split('-U')[1])
            # Формируем списко для последующего преобразования
            dict_unit = {}
            year = need_min_year
            for j in list_unit_from_key:
                if j not in dict_unit:
                    value_sum = int(j.split('.')[1]) if len(j.split('.')) > 1 else 0
                    year_tmp = value_sum + year
                    dict_unit[j] = str(year_tmp)

            # Чистим и преобразуем датафрейм с названиями измерений к дальнейшему мержу
            df_unit_tmp['year_transaction'] = df_unit_tmp.variable.apply(lambda x: dict_unit[str(x).split('-U')[1]])
            df_unit_tmp = df_unit_tmp.query('year_transaction not in @lst_query')
            df_unit_tmp.rename(columns={'value': 'units_value'}, inplace=True)
            df_unit_tmp = df_unit_tmp[['Product_code', 'Product_label', 'units_value', 'reporter_country', 
                                                      'partner_country', 'year_transaction']]

            # Чистим и преобразуем датафрейм с измерениями к дальнейшему мержу
            df_quantity_tmp.rename(columns={'variable': 'year_transaction'}, inplace=True)
            df_quantity_tmp['year_transaction'] =  df_quantity_tmp.year_transaction.apply(lambda x: x.split(' in ')[1])
            df_quantity_tmp = df_quantity_tmp.query('year_transaction not in @lst_query')

            # Мержим датасеты
            df_merge_quantities = df_quantity_tmp.merge(df_unit_tmp, on=['Product_code', 'year_transaction','reporter_country',
                                                             'partner_country'], how='left')
            # Редактируем наименования партнеров для мержа
            void_df_quantities = pd.concat((void_df_quantities, df_merge_quantities))
    
    # Сохраняем ошибки в файл       
    with open(f'{type_operation}_error_itc.json', 'w', encoding='utf-8') as file_json:
        json.dump(dct_error, file_json, indent=4, ensure_ascii=False)
        
    return void_df_quantities



In [16]:
# Класс для проверки загрузки данных в БД и их очистку
class Check_zero_in_db_value:
    
    def __init__(self, reporter_code, engine_class, df):
        """
        reporter_code: код репортера
        engine_class: движок подключения к БД
        df: полученный датафрейм на этапе обработки данных
        """
        self.reporter_code = reporter_code
        self.engine_class = engine_class
        self.df = df
    
    def get_count_rows(self, need_year=None):
        """
        need_year: минимально максимальный год из ранее загруженных данных в БД
        return: количество строк по конкретному репортеру в БД
        """
        # Если такой год присутствует, то вернуть рассчет без его учета
        # Для корректной валидации
        if need_year:
            return pd.read_sql(f"""SELECT COUNT(year) as count_rows 
                                   FROM tc 
                                   WHERE reporter_code = {self.reporter_code} AND year > {need_year}""",
                            con=self.engine_class).count_rows[0]
        # Если данных нет или они полностью совпадают, то посчитать все строки
        else:
            return pd.read_sql(f"""SELECT COUNT(year) as count_rows 
                                   FROM tc 
                                   WHERE reporter_code = {self.reporter_code}""",
                            con=self.engine_class).count_rows[0]
    
    def get_min_year_in_db(self, need_year):
        """
        need_year: минимальный год из датафрейма
        return: максимальный год из всех годов, которые меньше need_year или need_year если таковых нет
        """
        year_list = pd.read_sql(f"""SELECT DISTINCT(year) AS year 
                                    FROM tc
                                    WHERE reporter_code = {self.reporter_code} AND year < {need_year}
                                    ORDER BY year""",
                        con=self.engine_class).year
        return need_year if year_list.shape[0]  == 0 else year_list.max()
    
    def get_min_year_in_df(self):
        """
        return: минимальный год из датафрейма
        """
        return min([int(y) for y in self.df.year.unique()])
        
    def delete_need_value(self):
        """
        return: True если очистка прошла успешно, иначе False
        """
        if (self.get_min_year_in_df() == self.get_min_year_in_db(self.get_min_year_in_df())) or \
            (self.get_min_year_in_db(self.get_min_year_in_df()) > self.get_min_year_in_df()):
                
            with self.engine_class.cursor() as cr:
                cr.execute(f"""DELETE FROM tc 
                               WHERE reporter_code = {self.reporter_code}""")
                self.engine_class.commit()
            return self.get_count_rows() == 0
        
        elif self.get_min_year_in_db(self.get_min_year_in_df()) < self.get_min_year_in_df():

            with self.engine_class.cursor() as cr:
                cr.execute(f"""DELETE FROM tc 
                               WHERE reporter_code = {self.reporter_code} 
                               AND year > {self.get_min_year_in_db(self.get_min_year_in_df())}""")
                self.engine_class.commit()
            return self.get_count_rows(self.get_min_year_in_db(self.get_min_year_in_df())) == 0
        
    def check_value(self):
        if self.get_count_rows() == 0:
            return True
        return self.delete_need_value()

# Трансформация данных

In [17]:
# Для формирования путей к нужным папкам с файлами
type_flow_import = 'Imports'
type_flow_export = 'Exports'
value = 'Values'
qty = 'Quantities'

In [18]:
# "Cabo Verde", "South Africa", "Botswana", "Cambodia", "Guatemala", "Saudi Arabia", "Slovakia", "Slovenia",

In [19]:
# Подставить нужного репортера
reporter_name = "Slovenia"
# Флаг. Если в имени репортера присутствует and (Antigua_and_Barbuda) Присвоить значение True, иначе False
flag_and_in_reporter_name = False

In [20]:
# Проверка количесвта стран для экспорта (только на шаге парсинга TRADE_VALUE)
if os.path.exists('Imports_res.json'):
    with open('Imports_res.json', encoding='utf-8') as fl:
        js_imp = json.load(fl)
    if reporter_name in js_imp:
        print(len(js_imp[reporter_name]))

231


In [21]:
# Проверка количесвта стран для экспорта (только на шаге парсинга TRADE_VALUE)
if os.path.exists('Exports_res.json'):
    with open('Exports_res.json', encoding='utf-8') as fl:
        js_exp = json.load(fl)
    if reporter_name in js_exp:
        print(len(js_exp[reporter_name]))

223


In [22]:
# 'Imports' = 1 'Exports' = 2
# Ссылки для Импорта
path_values_import = os.path.join(os.getcwd(), f"""{reporter_name}_{type_flow_import}_{value}""")
path_quantities_import = os.path.join(os.getcwd(), f"""{reporter_name}_{type_flow_import}_{qty}""")

# Ссылки для Экспорта
path_values_export = os.path.join(os.getcwd(), f"""{reporter_name}_{type_flow_export}_{value}""")
path_quantities_export = os.path.join(os.getcwd(), f"""{reporter_name}_{type_flow_export}_{qty}""")

In [23]:
# Удаляем скаченные дубликаты, если такие есть
# Для trade_value Imports
for val in Path(path_values_import).glob("**/*(1)*.txt"):
    print(val)
    val.unlink()

# Для quantities Imports
for quant in Path(path_quantities_import).glob("**/*(1)*.txt"):
    print(quant)
    quant.unlink()

In [24]:
# Удаляем скаченные дубликаты, если такие есть
# Для trade_value Exports
for val in Path(path_values_export).glob("**/*(1)*.txt"):
    print(val)
    val.unlink()

# Для quantities Exports
for quant in Path(path_quantities_export).glob("**/*(1)*.txt"):
    print(quant)
    quant.unlink()

# Сборка для Импорт

In [25]:
# Датафрейм Импорт tarde_value
df_trade_value_import = trade_value_build(path_values_import, 1)

231it [00:19, 11.99it/s]


In [26]:
# Датафрейм Импорт quantities
df_quantities_import = quantities_build(type_flow_import, path_values_import, path_quantities_import)

231it [00:38,  5.96it/s]


In [27]:
# Собираем единый датасет trade_value + quantities
df_full_import = df_trade_value_import.merge(df_quantities_import, on=['Product_code', 'year_transaction','reporter_country',
                                                     'partner_country'], how='left', suffixes=('_trade', '_netweight'))

df_full_import = df_full_import[['Product_code', 'Product_label', 'value_trade', 'reporter_country', 'partner_country', 'trade_flow_code',
                   'classification', 'update_date', 'year_transaction', 'period', 'aggregate_level', 'flag', 'plus', 'load_mark',
                   'partner_code', 'reporter_code', 'name_itc', 'value_netweight', 'units_value']]

df_full_import.rename(columns={'Product_code': 'commodity_code', 'value_trade': 'trade_value', 'year_transaction': 'year',
                        'value_netweight': 'netweight'}, inplace=True)

In [28]:
# Проверим, какие измерения попали в датасет
df_full_import.units_value.unique()

array(['Tons', nan], dtype=object)

# Сборка для Экспорт

In [29]:
# Датафрейм Экспорт tarde_value
df_trade_value_export = trade_value_build(path_values_export, 2)

223it [00:16, 13.92it/s]


In [30]:
# Датафрейм Экспорт quantities
df_quantities_export = quantities_build(type_flow_export, path_values_export, path_quantities_export)

223it [00:36,  6.18it/s]


In [31]:
# Собираем единый датасет trade_value + quantities
df_full_export = df_trade_value_export.merge(df_quantities_export, on=['Product_code', 'year_transaction','reporter_country',
                                                     'partner_country'], how='left', suffixes=('_trade', '_netweight'))

df_full_export = df_full_export[['Product_code', 'Product_label', 'value_trade', 'reporter_country', 'partner_country', 'trade_flow_code',
                   'classification', 'update_date', 'year_transaction', 'period', 'aggregate_level', 'flag', 'plus', 'load_mark',
                   'partner_code', 'reporter_code', 'name_itc', 'value_netweight', 'units_value']]

df_full_export.rename(columns={'Product_code': 'commodity_code', 'value_trade': 'trade_value', 'year_transaction': 'year',
                        'value_netweight': 'netweight'}, inplace=True)

In [32]:
# Проверим, какие измерения попали в датасет
df_full_export.units_value.unique()

array(['Tons', nan], dtype=object)

### [⬅ Навигация](#Навигация)

# Сборка основного датасета

In [33]:
# Собираем все данные в один датафрейм
df_all_data = pd.concat((df_full_import, df_full_export))

In [34]:
def fix_d_izm(units: pd.Series, netweight: pd.Series) -> pd.Series:
    """
    Получает на вход две серии и исходя из условия получает приведенную к нужному формату qty

    :param units: колонку(серию) с типами измерения

    :param netweight: колонку(серию) со значением величины (вес)

    :return: преобразованную серию
    """
    coefficients = np.where(units.isin(['1000 meters', '1000 square meters']), 1000, 1)
    return netweight * coefficients

In [35]:
# Заменяем пропуски 
df_all_data['netweight'] = df_all_data.netweight.fillna(0)
df_all_data['units_value'] = df_all_data.units_value.fillna('без размерности')


# Заменяем на поправленные единицы измерения
df_all_data['netweight'] = fix_d_izm(df_all_data['units_value'], df_all_data['netweight'])

In [36]:
# Меняем названия величин измерений на русские
df_all_data['units_value'] = df_all_data.units_value.map(dict_deizm)
df_all_data.units_value.unique()

array(['килограмм', 'без размерности'], dtype=object)

In [37]:
# Разносим данные по qty и netweight
df_all_data['qty'] = np.where(df_all_data.units_value != 'килограмм', df_all_data.netweight, 0)
df_all_data['netweight'] = np.where(df_all_data.units_value == 'килограмм', df_all_data.netweight * 1000, 0)

In [38]:
df_all_data = df_all_data.merge(df_deizm, on='units_value', how='left')

In [39]:
# Выбираем нужные столбцы в нужном порядке
df_all_data = df_all_data[['classification', 'year', 'period', 'aggregate_level', 'trade_flow_code', 'reporter_code', 
                       'partner_code', 'commodity_code', 'qty_unit_code', 'qty', 'netweight', 
                       'trade_value', 'flag', 'plus', 'load_mark', 'update_date']]

In [40]:
df_all_data['update_date'] = datetime.now().strftime('%Y-%m-%d')
df_all_data['customs_proc_code'] = 'C00'
df_all_data['mode_of_transport_code'] = '0'
df_all_data['partner_code_2nd'] = 0
df_all_data.shape

(184134, 19)

In [41]:
df_all_data.isna().sum()

classification            0
year                      0
period                    0
aggregate_level           0
trade_flow_code           0
reporter_code             0
partner_code              0
commodity_code            0
qty_unit_code             0
qty                       0
netweight                 0
trade_value               0
flag                      0
plus                      0
load_mark                 0
update_date               0
customs_proc_code         0
mode_of_transport_code    0
partner_code_2nd          0
dtype: int64

### [⬅ Навигация](#Навигация)

# Сохраняем данные в БД

In [42]:
# Получаем код репортера
reporter_code_for_check = df_all_data.reporter_code.unique()[0]

In [43]:
# Перед записью в БД проверим года
sorted(df_all_data.year.unique().tolist())

['2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 '2022',
 '2023',
 '2024']

In [44]:
# Запись в партиции БД
if Check_zero_in_db_value(reporter_code_for_check, engine, df_all_data).check_value():
    print(f'Загружаем репортера {reporter_code_for_check}')
    for year_cycle in df_all_data.year.unique().tolist():

        df_data_in_bd = df_all_data.query('year == @year_cycle')
        print(f'{year_cycle}, количество строк: {df_data_in_bd.shape[0]}')
        df_data_in_bd.to_sql(f'_{year_cycle}', 
                             con=conn, 
                             schema='', 
                             if_exists='append', index=False)
else:
    print('Старые данные не очищенны')

Загружаем репортера 705
2013, количество строк: 13864
2014, количество строк: 13837
2015, количество строк: 13868
2016, количество строк: 14457
2017, количество строк: 15232
2018, количество строк: 15296
2022, количество строк: 17114
2023, количество строк: 17114
2019, количество строк: 15480
2020, количество строк: 15594
2021, количество строк: 16402
2024, количество строк: 15876


In [45]:
Check_zero_in_db_value(reporter_code_for_check, engine, df_all_data).get_min_year_in_db(df_all_data.year.min())

'2013'

In [46]:
# Если данные уже были в базе передать в параметр get_count_rows год
# Пример, у нас в датафрейме минимальный год 2011, но в базе есть 2009 и 2010 года (их мы не обновляем)
# Для корретной валидации нужно передать 2010 год(максимальный год, который есть в базе, но нет в датафрейме)
df_all_data.shape[0] ==  Check_zero_in_db_value(reporter_code_for_check, engine, df_all_data).get_count_rows(2012)

True

-------

### [⬅ Навигация](#Навигация)